In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
stream_inv_line_df = spark.readStream.table("invoice_db.invoice_line_data_stg")

In [0]:
stream_inv_line_df.display()

invoice_number,unique_invoice_identifier,account_number,customer_name,invoice_line_identifier,invoice_line_amount,price,invoice_start_date,invoice_end_date,invoice_total_amount,invoice_status,target_committime,load_month,load_date,batch_id
123462837-SS,f237165c-6dc2-4695-9635-6245f8913c54,123-456-658,Joseph Christ,1,200.00,200.00,2024-05-18,2024-05-31,450.00,ADD,2024-07-21T07:58:04.651+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-07_58_04
123462837-SS,f237165c-6dc2-4695-9635-6245f8913c54,123-456-658,Joseph Christ,2,200.00,200.00,2024-05-19,2024-06-01,450.00,ADD,2024-07-21T07:58:04.651+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-07_58_04
123462835-SS,f237165c-6dc2-4695-9635-6245f8913c65,123-456-680,Alpha,1,600.00,200.00,2024-05-20,2024-05-31,750.00,ADD,2024-07-21T08:30:27.614+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-08_30_27
123462869-SS,l237165c-6dc2-4695-9635-6245f8913g57,123-456-659,Jane Christ,2,200.00,250.00,2024-05-20,2024-06-04,850.00,EDIT,2024-07-21T08:30:27.614+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-08_30_27
123462869-SS,l237165c-6dc2-4695-9635-6245f8913g57,123-456-659,Jane Christ,1,300.00,200.00,2024-05-20,2024-05-31,750.00,EDIT,2024-07-21T08:30:27.614+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-08_30_27
123462838-SS,f237165c-6dc2-4695-9635-6245f8913c54,123-456-659,Jane Christ,2,200.00,250.00,2024-05-20,2024-06-04,850.00,ADD,2024-07-21T08:19:21.900+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-08_19_21
123462838-SS,f237165c-6dc2-4695-9635-6245f8913c54,123-456-659,Jane Christ,1,300.00,200.00,2024-05-20,2024-05-31,750.00,ADD,2024-07-21T08:19:21.900+0000,2024-07-01,2024-07-21,BATCH-2024-07-21-08_19_21


In [0]:
aggregated_df = stream_inv_line_df \
    .groupBy("account_number", "customer_name","invoice_line_identifier") \
    .agg(
        expr("sum(invoice_line_amount) as total_line_amt"),
        expr("sum(price) as total_price"),
        expr("sum(invoice_total_amount) as total_invoice_amt")
    ) \
    .withColumn("target_committime", current_timestamp())

In [0]:
aggregated_df.display()

account_number,customer_name,invoice_line_identifier,total_line_amt,total_price,total_invoice_amt,target_committime
123-456-658,Joseph Christ,2,200.00,200.00,450.00,2024-07-21T11:25:51.202+0000
123-456-659,Jane Christ,2,400.00,500.00,1700.00,2024-07-21T11:25:51.202+0000
123-456-658,Joseph Christ,1,200.00,200.00,450.00,2024-07-21T11:25:51.202+0000
123-456-680,Alpha,1,600.00,200.00,750.00,2024-07-21T11:25:51.202+0000
123-456-659,Jane Christ,1,600.00,400.00,1500.00,2024-07-21T11:25:51.202+0000


In [0]:
def update_table(line_total_df,batch_id):
    line_total_df.createOrReplaceTempView("line_total_temp")
    merge_statement = """merge into invoice_db.invoice_total_line_data_stg t using line_total_temp s
    on t.account_number == s.account_number and t.customer_name == s.customer_name and t.invoice_line_identifier == s.invoice_line_identifier
    when matched then
    update set
    t.total_line_amt = t.total_line_amt + s.total_line_amt, 
    t.total_price = t.total_price + s.total_price, 
    t.total_invoice_amt = t.total_invoice_amt + s.total_invoice_amt,
    t.target_committime = s.target_committime
    when not matched then
    insert *
    """
    line_total_df._jdf.sparkSession().sql(merge_statement)

In [0]:
query = aggregated_df.writeStream \
    .foreachBatch(update_table) \
    .outputMode("update") \
    .option("checkpointLocation", "/FileStore/tables/invoices/invoice_total_line_data_chkpt") \
    .start()

query.awaitTermination()